# Рекомендация тарифов

Оператор сотовой связи предоставляет данные о поведении клиентов, которые используют новые тарифы smart и ultra. Однако часть клиентов компании до сих пор пользуется устаревшими тарифами. Наша задача - построить модель классификации, которая выберет подходящий тариф для пользователя. 

## Открываем и изучаем файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Читаем и изучаем файл

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('C:\\Users\\Дмитрий\\Desktop\\Проекты Я.Практикум\\tariff_recommendation\\users_behavior.csv')

In [3]:
display(df)
display(df.isna().mean())
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


calls       0.0
minutes     0.0
messages    0.0
mb_used     0.0
is_ultra    0.0
dtype: float64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB



Так как тествой выборки у нас нет, то разбиваем исходный датафрейм на тренировочную, валидационную и тестовую выборки в соотношении 3:1:1.

## Разбиваем данные на выборки

Разбиваем данные на выборки

In [4]:
df_train, df_valid_test = train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=12345)
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(1928, 5)
(643, 5)
(643, 5)


## Исследуем модели

Выделяем целевой признак и остальные

In [5]:
feature_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
feature_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
feature_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

### Дерево решений

In [6]:
best_model_1 = None
best_result_1 = 0
best_depth_1 = 0
for depth in range(1,11):
    model_1 = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_1.fit(feature_train, target_train)
    predictions_valid_1 = model_1.predict(feature_valid)
    result_1 = accuracy_score(target_valid, predictions_valid_1)
    if result_1 > best_result_1:
        best_model_1 = model_1
        best_result_1 = result_1
        best_depth_1 = depth  
        
print("best_depth =", best_depth_1, ": ", end='')
print(best_result_1) 

best_depth = 3 : 0.7853810264385692


### Случайный лес

In [7]:
best_model_2 = None
best_result_2 = 0
best_depth_2 = 0
for est in range(1, 51):
    for depth in range(1,11):
        model_2 = RandomForestClassifier(random_state=12345, 
                                         n_estimators=est, 
                                         max_depth=depth, 
                                         n_jobs=-1)  
        model_2.fit(feature_train, target_train)  
        predictions_valid_2 = model_2.predict(feature_valid)
        result_2 = accuracy_score(target_valid, predictions_valid_2)  
        if result_2 > best_result_2:
            best_model_2 = model_2
            best_result_2 = result_2
            best_est_2 = est
            best_depth_2 = depth

print("best_est =", best_est_2)
print("best_depth =", best_depth_2)
print(best_result_2) 

best_est = 40
best_depth = 8
0.8087091757387247


### Логистическая регрессия

In [8]:
model_3 = LogisticRegression(random_state=12345)
model_3.fit(feature_train, target_train) 
predictions_valid_3 = model_3.predict(feature_valid)
result_3 = accuracy_score(target_valid, predictions_valid_3)  
print('Accuracy:', result_3)

Accuracy: 0.7107309486780715


### Выводы

* Наибольшую точность из всех моделей показала модель случайного леса с 40 деревьями и глубиной 8: accuracy = 0.8087 (исследовались модели до 50 деревьев и глубины равной 10).

* Модель "решающее дерево" показала свой лучший результат при глубине = 3: accuracy = 0.7854 (исследовались модели с глубиной до 10).

* Модель логистической регрессии показала худший результат, равный 0.7589.

## Тестируем модель

In [9]:
predictions_test_2 = best_model_2.predict(feature_test)
result_test_2 = accuracy_score(target_test, predictions_test_2)  
print(result_test_2)

0.7962674961119751


## Вывод

* Полученные данные изучили и разбили на обучающую, валидационную и тестовую в соотношении 3:1:1.

* Исследовали модели решающего дерева, случайного леса и логистической регрессии, меняя гиперпараметры в первых двух.
Проверили модели на валидационной выборке.Лучшее значение accuracy показала модель случайного леса с 40 деревьями и глубиной 8 (0.8087). Далее следовали модели решающего дерева с глубиной 3 (0.7854) и логистической регрессии (0.7589). 

* Далее проверили лучшие модели на тестовой выборке. Значение accuracy незначительно упало и составило 0.7963. Модель не переобучилась и обладает достаточной для заказчика точностью.